In [1]:
from gurobipy import *
from prettytable import PrettyTable
from math import pi
import numpy as np

In [2]:
OPF = Model('3 bus OPF')

In [3]:
#Sets
G = 3 # Generators
B = 3 # Bus
L = 3 #Lines
Gen = range(G)
Bus = range(B)
Line = range(L)

In [4]:
X = {(0,1):10, (0,2):8, (1,2):5}
L = {0:200, 1:550, 2:100} # electircal Load (MW)
Sbase = 100 # Power Base (MVA) 

In [5]:
GenAt = {Gen[0]:{'a':0.001562 ,'b':7.92 ,'c':561, 'Pmin':150 ,'Pmax':600},
         Gen[1]:{'a':0.001940 ,'b':7.85 ,'c':310, 'Pmin':100 ,'Pmax':400},
         Gen[2]:{'a':0.004820 ,'b':7.97 ,'c':78,  'Pmin':50 , 'Pmax':200}}

In [6]:
Pmin = [(GenAt[i]['Pmin'])/Sbase for i in Gen]
Pmax = [(GenAt[i]['Pmax'])/Sbase for i in Gen]
g = [a for a in Gen]
b = [a for a in Bus]

In [7]:
P = OPF.addVars(g, lb = Pmin, ub = Pmax, name = 'Gen')
Delta = OPF.addVars(b, lb = -pi, ub = pi, name = 'PhaseAngle')
Pij = OPF.addVars(X, name = 'line')

In [8]:
BusAngle = OPF.addConstrs((Pij[(i,j)] == (Delta[i]-Delta[j])*X[(i,j)]*Sbase for (i,j) in Pij.keys()), name = 'BusAngle')
lineFlow = OPF.addConstrs((P[i] + sum(Pij[(i,j)],) == L[i] for (i,j) in Pij.keys()), name = 'lineFlow')

Delta[0].lb = 0
Delta[0].ub = 0

In [9]:
OF = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)

In [10]:
OPF.setObjective(OF, GRB.MINIMIZE)
OPF.Params.Method=-1
OPF.setParam( 'OutputFlag', True )
OPF.optimize()



status = OPF.status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
or status == GRB.Status.UNBOUNDED:
    OPF.computeIIS()
    OPF.write("model.ilp")

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 6 rows, 9 columns and 15 nonzeros
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 8e+00]
  QObjective range [3e-03, 1e-02]
  Bounds range     [5e-01, 6e+00]
  RHS range        [2e+02, 6e+02]
Presolve removed 5 rows and 7 columns
Presolve time: 0.01s

Barrier solved model in 0 iterations and 0.01 seconds
Model is infeasible
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.4900000e+02   4.705310e+02   0.000000e+00      0s

IIS computed: 6 constraints and 2 bounds
IIS runtime: 0.00 seconds


In [11]:
OPF.printStats()


Statistics for model 3 bus OPF :
  Linear constraint matrix    : 6 Constrs, 9 Vars, 15 NZs
  Matrix coefficient range    : [ 1, 1000 ]
  Objective coefficient range : [ 7.85, 7.97 ]
  Variable bound range        : [ 0.5, 6 ]
  RHS coefficient range       : [ 200, 550 ]
